<a href="https://colab.research.google.com/github/samiha-mahin/Ovarian-cancer/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle tensorflow numpy pandas opencv-python matplotlib scikit-learn

In [2]:
from google.colab import files
files.upload()  # Upload your kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samihamuntahamahin","key":"1ad5aaba9143fbd38da418ad8d278398"}'}

In [3]:

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets list -s "ovarian cancer"

ref                                                             title                                                      size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
saurabhshahane/predict-ovarian-cancer                           Predict Ovarian Cancer                                   322564  2021-02-06 08:15:47.360000           2860         47  0.7058824        
yoshifumimiya/6-ovarian-cancer-datasets                         7 ovarian cancer datasets                             239456735  2024-04-21 13:37:49.933000           3475         48  0.7647059        
sunilthite/ovarian-cancer-classification-dataset                Ovarian Cancer Subtype Classification                3554471607  2023-10-31 08:34:17.390000           1533         38  1.0          

In [5]:
!kaggle datasets download -d bitsnpieces/ovarian-cancer-and-subtypes-dataset-histopathology --unzip

Dataset URL: https://www.kaggle.com/datasets/bitsnpieces/ovarian-cancer-and-subtypes-dataset-histopathology
License(s): CC-BY-SA-4.0


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split

# Define dataset path
DATASET_PATH = "OvarianCancer"

# Define classes
classes = ["Mucinous", "Non_Cancerous", "Endometri", "Serous", "Clear_Cell"]
class_mapping = {class_name: i for i, class_name in enumerate(classes)}

# Image size for ResNet50
IMG_SIZE = 224

# Lists to store images & labels
X, y = [], []

# Load images
for class_name in classes:
    class_path = os.path.join(DATASET_PATH, class_name)
    if not os.path.exists(class_path):
        continue
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        X.append(img)
        y.append(class_mapping[class_name])

# Convert to NumPy arrays
X = np.array(X, dtype=np.float32) / 255.0  # Normalize pixels
y = to_categorical(y, num_classes=len(classes))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dataset loaded: {len(X)} images")

# Load pre-trained ResNet50 with frozen layers
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze first 140 layers
for layer in base_model.layers[:140]:
    layer.trainable = False

# Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(classes), activation="softmax")(x)

# Create model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile model with SGD optimizer
model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9),
              loss="categorical_crossentropy", metrics=["accuracy"])

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
train_generator = datagen.flow(X_train, y_train, batch_size=16)

# Train the model
history = model.fit(train_generator, validation_data=(X_test, y_test), epochs=30)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")
print(f"Test Loss: {loss:.4f}")

# Save the model
model.save("resnet50_ovarian_cancer_model.h5")
print("Model saved successfully!")

# Plot accuracy & loss
def plot_history(history):
    plt.figure(figsize=(12, 5))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label="Train Accuracy")
    plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
    plt.title("Accuracy Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.title("Loss Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.show()

plot_history(history)


Dataset loaded: 497 images


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 166s 6s/step - accuracy: 0.2880 - loss: 2.6140 - val_accuracy: 0.1600 - val_loss: 1.9801
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 139s 6s/step - accuracy: 0.3380 - loss: 2.0128 - val_accuracy: 0.2500 - val_loss: 2.9858
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 141s 6s/step - accuracy: 0.2932 - loss: 1.7149 - val_accuracy: 0.2700 - val_loss: 2.6133
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 148s 6s/step - accuracy: 0.3806 - loss: 1.4955 - val_accuracy: 0.2700 - val_loss: 2.2219
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 144s 6s/step - accuracy: 0.4277 - loss: 1.4044 - val_accuracy: 0.2500 - val_loss: 1.7769
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 202s 6s/step - accuracy: 0.3585 - loss: 1.5105 - val_accuracy: 0.2500 - val_loss: 1.9079
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 145s 6s/step - accuracy: 0.4168 - loss: 1.5261 - val_accuracy: 0.3300 - val_loss: 1.6077
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 138s 6s/step - accuracy: 0.3394 - loss: 1.4489 - val_accuracy: 0.3500 - v